# Migrating music from Spotify to Google Play
it seems there isn't an easy option out there if you want to switch between subsctription services. This really sucks because it makes it really hard to leave. GP or iTunes might be better, but there's just no way I'd be willing to find all of my music again manually. It's actually easier to pirate a library than it is to get it back on another streaming service. 

This notebook hopes to solve this problem

resorces used:
* https://spotipy.readthedocs.io/en/latest/#authorized-requests
* https://unofficial-google-music-api.readthedocs.io/en/latest/usage.html#usage
* https://github.com/suihanki/gmusic-import/blob/master/gmusic-import-script.py

also, keeping all the imports below here. some stuff wont work if they're not installed

In [13]:
import sys
import spotipy
import spotipy.util as util
import gmusicapi as gm
import json
import pandas as pd


#### get the spotify token

https://developer.spotify.com/dashboard/applications

In [14]:

SPOTIPY_CLIENT_ID ='xxx'
SPOTIPY_CLIENT_SECRET='xxx'
SPOTIPY_REDIRECT_URI='http://localhost/'
scope = 'user-library-read'
username = 'bashsebbash'

token = util.prompt_for_user_token(username,scope,SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET,SPOTIPY_REDIRECT_URI)

### get songs in Spotify library

In [15]:
i = 0
n = 40

spotify_lib = []
while i < 10000:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks(limit=n, offset=i)
    songs = {
            'id':  [y['track']['external_ids']['isrc'] for y in [x for x in results['items']]],
             'name':[y['track']['name'] for y in [x for x in results['items']]],
            'track_number':[y['track']['track_number'] for y in [x for x in results['items']]],
            'album':[y['track']['album']['name'] for y in [x for x in results['items']]],
            'artist':[str([x['name'] for x in y['track']['artists']]) for y in [x for x in results['items']]],
            'popularity':[y['track']['popularity'] for y in [x for x in results['items']]],
            'uri':   [y['track']['uri'] for y in [x for x in results['items']]]
            }
    songs = pd.DataFrame(songs)
    i+= (n-1)
    spotify_lib.append(songs)


In [16]:
spotify_lib = pd.concat(spotify_lib)

In [84]:

# spotify_lib.artist = spotify_lib.artist.str.replace('\[|\]|\'|\"','').str.split(',')
formatted = spotify_lib[['name','artist','album','track_number']].copy(deep=True)
formatted = formatted.reset_index(drop = True)
formatted['Artist'] = formatted.artist.apply(lambda x:x)


In [125]:
formatted[['name','Artist','album','track_number']].to_csv('songs.csv')

### connect to google play

In [34]:
a = gm.clients.Mobileclient()
a.login(email = 'xxx@gmail.com', password = 'password',android_id = '10F0056DABBE')


In [86]:
formatted['storeId']= None

### get the storeIDs for each song

In [95]:

for index, row in formatted.iterrows():
    try:
        res = a.search(row['name']+' '+row['album']+' '+row['Artist'][0])
        formatted['storeId'][index] = res['song_hits'][0]['track']['storeId']
    except:
        pass
    

C:\Users\Sebas\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Sebas\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Sebas\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2605' in position 445: character maps to <undefined>
Call stack:
  File "C:\Users\Sebas\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Sebas\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Sebas\Anaconda3\lib\site-packages\ipykernel_launcher.py", l

In [106]:
formatted.to_csv('songs.csv')

### add them all in google play music

In [105]:
for x in list(formatted.storeId):
    try:
        a.add_store_tracks(x)
    except:
        pass

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Sebas\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Sebas\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u030a' in position 138: character maps to <undefined>
Call stack:
  File "C:\Users\Sebas\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Sebas\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Sebas\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Sebas\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Sebas\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_l

In [103]:
formatted

,name,artist,album,track_number,Artist,storeId
0,The Anthem,[Myke Bogan],Silk Jockstrap,6.0,[Myke Bogan],T6koip4uh5isnz4covsajifhpkm
1,Me Gustas Tu,[Manu Chao],Próxima Estación: Esperanza,6.0,[Manu Chao],T7r3qdfgqorsijomcrdblc552yu
2,Gasoline,[Prof],Liability,16.0,[Prof],T5gdacdcrxunvlzuoosl5eczway
3,Slow Burn,[Doomtree],False Hopes,7.0,[Doomtree],T2f7om2v4jaqhs2f4qbb2audscy
4,All Your Light (Times Like These),[Portugal. The Man],In The Mountain In The Cloud,8.0,[Portugal. The Man],T5npbrnosql3ckjw4n5oiamo7xm
5,The Ringer,[Eminem],Kamikaze,1.0,[Eminem],Twpz67tpoe4f3rsfeu4guqo2jii
6,Greatest,[Eminem],Kamikaze,2.0,[Eminem],Td43dsxg6ij262bke2cz6jbv22y
7,Lucky You (feat. Joyner Lucas),"[Eminem, Joyner Lucas]",Kamikaze,3.0,"[Eminem, Joyner Lucas]",Tea2hr6ed6yjeoix44jrcugsaf4
8,Paul - Skit,[Paul Rosenberg],Kamikaze,4.0,[Paul Rosenberg],Tksyzc46znh7yuuwzn3ia7vcigm
9,Normal,[Eminem],Kamikaze,5.0,[Eminem],T3nwz55t3fwgxci5p3wm765xakm


In [22]:
results['items'][0]['track']

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5BLrsIX3Uipmt9QAiZGYEl'},
    'href': 'https://api.spotify.com/v1/artists/5BLrsIX3Uipmt9QAiZGYEl',
    'id': '5BLrsIX3Uipmt9QAiZGYEl',
    'name': 'Myke Bogan',
    'type': 'artist',
    'uri': 'spotify:artist:5BLrsIX3Uipmt9QAiZGYEl'}],
  'available_markets': ['AD',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'EC',
   'EE',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IS',
   'IT',
   'JP',
   'LI',
   'LT',
   'LU',
   'LV',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'PA',
   'PE',
   'PH',
   'PL',
   'PT',
   'PY',
   'RO',
   'SE',
   'SG',
   'SK',
   'SV',
   'TH',
   'TR',
   'TW',
   'US',
   'UY',
   'VN',
   'ZA'],
  'external_urls': {'spotify': 'https://open.

In [57]:
spotify_lib

,album,artist,id,name,popularity,track_number,uri
0,Silk Jockstrap,[Myke Bogan],USUYG1050565,The Anthem,34.0,6.0,spotify:track:6fW5spjBKrpBFI2mhvMvh5
1,Próxima Estación: Esperanza,[Manu Chao],FR84R0000004,Me Gustas Tu,71.0,6.0,spotify:track:6b37xrsNCWYIUphFBazqD6
2,Liability,[Prof],USBWK1500028,Gasoline,46.0,16.0,spotify:track:7nWkB7u5QT1JblURO10NYK
3,False Hopes,[Doomtree],USTC40895195,Slow Burn,43.0,7.0,spotify:track:3NbJl85bcTrbpfovfezTYG
4,In The Mountain In The Cloud,[Portugal. The Man],USAT21100258,All Your Light (Times Like These),53.0,8.0,spotify:track:3UOlYczxtt70Xj1VplXgj8
5,Kamikaze,[Eminem],USUM71813329,The Ringer,91.0,1.0,spotify:track:2jt2WxXMCD4zjACthkJQVE
6,Kamikaze,[Eminem],USUM71813333,Greatest,88.0,2.0,spotify:track:2XTquzYQAdT1Hk78bOUwsv
7,Kamikaze,"[Eminem, Joyner Lucas]",USUM71813334,Lucky You (feat. Joyner Lucas),96.0,3.0,spotify:track:60SdxE8apGAxMiRrpbmLY0
8,Kamikaze,[Paul Rosenberg],USUM71813335,Paul - Skit,82.0,4.0,spotify:track:0Uv5Kp8sEnriJjkX4oLfNI
9,Kamikaze,[Eminem],USUM71813336,Normal,85.0,5.0,spotify:track:0evt4UZbdhnHtcAnxkm6A1
